# 7. BIST50 Transformer Modelleri - Tek Hisse (DÜZELTİLMİŞ)

**Modeller:** Informer, Autoformer, TimesNet, DeepAR, TFT, TSMixer (NeuralForecast)

**Veri:** Tek hisse × 250 gün (Az Veri)

**Düzeltme:** Her periyot için AYRI model eğitiliyor (1, 10, 21 gün)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Gerekli kütüphaneleri kur
!pip install -q neuralforecast pytorch-lightning

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from math import sqrt
import torch
import os

# NeuralForecast
from neuralforecast import NeuralForecast
from neuralforecast.models import Informer, Autoformer, TimesNet, DeepAR, TFT, TSMixer

plt.style.use('fivethirtyeight')

# GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

# Parametreler
TEST_SIZE = 21
INPUT_SIZE = 30

# Tahmin periyotları - HER BİRİ İÇİN AYRI MODEL EĞİTİLECEK
TAHMIN_PERIYOTLARI = {
    '1_gun': 1,
    '10_gun': 10,
    '21_gun': 21
}

# Yollar
DATA_FILE = "/content/drive/MyDrive/Colab Notebooks/tez/Data/BIST_50_2018_Data.csv"
RESULTS_DIR = '/content/drive/MyDrive/Colab Notebooks/Sonuclar/7_Transformers'
os.makedirs(RESULTS_DIR, exist_ok=True)

NOTEBOOK_ADI = "7_Transformers"
VERI_TIPI = "az_veri"
KATEGORI = "Transformer"

print(f"Veri: {DATA_FILE}")
print(f"Sonuç Dizini: {RESULTS_DIR}")

In [ ]:
# Güvenli metrik hesaplama
def calculate_metrics_safe(y_true, y_pred, model_name):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    nan_count = np.isnan(y_pred).sum()
    if nan_count > 0:
        print(f"  {nan_count} NaN, interpolasyon...")
    y_pred = pd.Series(y_pred).interpolate().bfill().ffill().values

    valid = ~np.isnan(y_pred) & ~np.isnan(y_true)
    if valid.sum() == 0:
        return np.nan, np.nan, np.nan, np.nan, y_pred

    rmse = sqrt(mean_squared_error(y_true[valid], y_pred[valid]))
    mae = mean_absolute_error(y_true[valid], y_pred[valid])
    r2 = r2_score(y_true[valid], y_pred[valid])
    mape = mean_absolute_percentage_error(y_true[valid], y_pred[valid]) * 100
    return rmse, mae, r2, mape, y_pred

In [ ]:
# Veri yükle
print("Veri yükleniyor...")
df_raw = pd.read_csv(DATA_FILE)

if 'Now' in df_raw.columns:
    df_raw = df_raw.rename(columns={'Now': 'Close'})

df_raw['Date'] = pd.to_datetime(df_raw['Date'], format='%d.%m.%Y')

if df_raw['Close'].dtype == 'object':
    df_raw['Close'] = df_raw['Close'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

df = df_raw[['Date', 'Close']].dropna().sort_values('Date').reset_index(drop=True)

print(f"Toplam: {len(df)} gün")
print(f"Tarih: {df['Date'].min()} - {df['Date'].max()}")

# Train/Test
train_df = df.iloc[:-TEST_SIZE].copy()
test_df = df.iloc[-TEST_SIZE:].copy()

print(f"Train: {len(train_df)}, Test: {len(test_df)}")

# NeuralForecast formatı
train_nf = train_df.rename(columns={'Date': 'ds', 'Close': 'y'})
train_nf['unique_id'] = 'BIST50'
train_nf = train_nf[['unique_id', 'ds', 'y']]

print(f"NeuralForecast format hazır: {train_nf.shape}")

In [ ]:
# =============================================================================
# HER PERİYOT İÇİN AYRI MODEL EĞİTİMİ
# =============================================================================
print("=" * 80)
print(" HER PERİYOT İÇİN AYRI MODEL EĞİTİMİ")
print("=" * 80)

tum_sonuclar = []

for periyot_adi, horizon in TAHMIN_PERIYOTLARI.items():
    print(f"\n{'='*70}")
    print(f" {periyot_adi.upper()} ({horizon} gün) için model eğitimi başlıyor...")
    print('='*70)

    # Bu periyot için gerçek değerler
    y_true = test_df['Close'].values[:horizon]

    # Model listesi - her periyot için yeniden oluştur
    model_configs = {
        'Informer': lambda h: Informer(h=h, input_size=INPUT_SIZE, hidden_size=64, n_head=8,
                                        learning_rate=1e-3, max_steps=500, batch_size=32,
                                        random_seed=42, scaler_type='standard'),
        'Autoformer': lambda h: Autoformer(h=h, input_size=INPUT_SIZE, hidden_size=64, n_head=8,
                                           learning_rate=1e-3, max_steps=500, batch_size=32,
                                           random_seed=42, scaler_type='standard'),
        'TimesNet': lambda h: TimesNet(h=h, input_size=INPUT_SIZE, hidden_size=64,
                                        learning_rate=1e-3, max_steps=500, batch_size=32,
                                        random_seed=42, scaler_type='standard'),
        'DeepAR': lambda h: DeepAR(h=h, input_size=INPUT_SIZE, lstm_n_layers=2, lstm_hidden_size=64,
                                    learning_rate=1e-3, max_steps=500, batch_size=32,
                                    random_seed=42, scaler_type='standard'),
        'TFT': lambda h: TFT(h=h, input_size=INPUT_SIZE, hidden_size=64, n_head=8,
                             learning_rate=1e-3, max_steps=500, batch_size=32,
                             random_seed=42, scaler_type='standard'),
        'TSMixer': lambda h: TSMixer(h=h, input_size=INPUT_SIZE, n_series=1, n_block=2, ff_dim=64,
                                      learning_rate=1e-3, max_steps=500, batch_size=32,
                                      random_seed=42, scaler_type='standard')
    }

    for model_name, model_fn in model_configs.items():
        print(f"\n   {model_name} eğitiliyor ({periyot_adi}, horizon={horizon})...")

        try:
            # Model oluştur (bu periyot için)
            model = model_fn(horizon)

            # Eğit
            nf = NeuralForecast(models=[model], freq='D')
            nf.fit(df=train_nf)

            # Tahmin
            pred = nf.predict().reset_index()
            y_pred = pred[model_name].values[:len(y_true)]

            # Metrikler
            rmse, mae, r2, mape, y_pred = calculate_metrics_safe(y_true, y_pred, model_name)

            print(f"      {model_name} ({periyot_adi}): RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")

            tum_sonuclar.append({
                'Notebook': NOTEBOOK_ADI,
                'Model': model_name,
                'Kategori': KATEGORI,
                'Veri_Tipi': VERI_TIPI,
                'Periyot': periyot_adi,
                'Periyot_Gun': horizon,
                'RMSE': round(rmse, 4) if not np.isnan(rmse) else np.nan,
                'MAE': round(mae, 4) if not np.isnan(mae) else np.nan,
                'R2': round(r2, 4) if not np.isnan(r2) else np.nan,
                'MAPE': round(mape, 4) if not np.isnan(mape) else np.nan
            })

        except Exception as e:
            print(f"      {model_name} ({periyot_adi}): HATA - {str(e)[:100]}")
            tum_sonuclar.append({
                'Notebook': NOTEBOOK_ADI,
                'Model': model_name,
                'Kategori': KATEGORI,
                'Veri_Tipi': VERI_TIPI,
                'Periyot': periyot_adi,
                'Periyot_Gun': horizon,
                'RMSE': np.nan,
                'MAE': np.nan,
                'R2': np.nan,
                'MAPE': np.nan
            })

sonuc_df = pd.DataFrame(tum_sonuclar)
print(f"\n Toplam {len(sonuc_df)} test tamamlandı!")

In [ ]:
# =============================================================================
# SONUÇ TABLOLARI
# =============================================================================
if len(sonuc_df) > 0:
    print("\n" + "=" * 80)
    print(" RMSE TABLOSU (Model × Periyot)")
    print("=" * 80)
    pivot_rmse = sonuc_df.pivot_table(index='Model', columns='Periyot', values='RMSE', aggfunc='first')
    if '1_gun' in pivot_rmse.columns:
        pivot_rmse = pivot_rmse[['1_gun', '10_gun', '21_gun']]
    print(pivot_rmse.round(4).to_string())

    print("\n" + "=" * 80)
    print(" MAE TABLOSU (Model × Periyot)")
    print("=" * 80)
    pivot_mae = sonuc_df.pivot_table(index='Model', columns='Periyot', values='MAE', aggfunc='first')
    if '1_gun' in pivot_mae.columns:
        pivot_mae = pivot_mae[['1_gun', '10_gun', '21_gun']]
    print(pivot_mae.round(4).to_string())

    print("\n" + "=" * 80)
    print(" R² TABLOSU (Model × Periyot)")
    print("=" * 80)
    pivot_r2 = sonuc_df.pivot_table(index='Model', columns='Periyot', values='R2', aggfunc='first')
    if '1_gun' in pivot_r2.columns:
        pivot_r2 = pivot_r2[['1_gun', '10_gun', '21_gun']]
    print(pivot_r2.round(4).to_string())

    print("\n" + "=" * 80)
    print(" MAPE TABLOSU (Model × Periyot)")
    print("=" * 80)
    pivot_mape = sonuc_df.pivot_table(index='Model', columns='Periyot', values='MAPE', aggfunc='first')
    if '1_gun' in pivot_mape.columns:
        pivot_mape = pivot_mape[['1_gun', '10_gun', '21_gun']]
    print(pivot_mape.round(4).to_string())

    # Kontrol: Periyotlar farklı sonuç veriyor mu?
    print('\n KONTROL: Periyotlar farklı sonuç veriyor mu?')
    for model in pivot_rmse.index:
        vals = pivot_rmse.loc[model].values
        if len(set([round(v, 4) for v in vals if not np.isnan(v)])) <= 1:
            print(f'   {model}: AYNI veya eksik - KONTROL ET!')
        else:
            print(f'   {model}: FARKLI - OK')

In [ ]:
# =============================================================================
# KARŞILAŞTIRMA GRAFİKLERİ
# =============================================================================
if len(sonuc_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'{NOTEBOOK_ADI} - 3 Periyot Karşılaştırması ({VERI_TIPI})', fontsize=14, fontweight='bold')

    pivot_rmse = sonuc_df.pivot_table(index='Model', columns='Periyot', values='RMSE', aggfunc='first')
    pivot_mae = sonuc_df.pivot_table(index='Model', columns='Periyot', values='MAE', aggfunc='first')
    pivot_r2 = sonuc_df.pivot_table(index='Model', columns='Periyot', values='R2', aggfunc='first')
    pivot_mape = sonuc_df.pivot_table(index='Model', columns='Periyot', values='MAPE', aggfunc='first')

    models = pivot_rmse.index.tolist()
    x = np.arange(len(models))
    width = 0.25
    colors = ['#e74c3c', '#f39c12', '#27ae60']
    periyotlar = ['1_gun', '10_gun', '21_gun']

    for ax, pivot, title, ylabel in [(axes[0,0], pivot_rmse, 'RMSE', 'RMSE'),
                                      (axes[0,1], pivot_mae, 'MAE', 'MAE'),
                                      (axes[1,0], pivot_r2, 'R²', 'R²'),
                                      (axes[1,1], pivot_mape, 'MAPE (%)', 'MAPE')]:
        for i, periyot in enumerate(periyotlar):
            if periyot in pivot.columns:
                ax.bar(x + i*width, pivot[periyot].values, width, label=periyot, color=colors[i])
        ax.set_title(f'{title} Karşılaştırması', fontweight='bold')
        ax.set_ylabel(ylabel)
        ax.set_xticks(x + width)
        ax.set_xticklabels(models, rotation=45, ha='right', fontsize=8)
        ax.legend()
        ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/{NOTEBOOK_ADI}_3periyot_karsilastirma.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\n Grafik kaydedildi: {RESULTS_DIR}/{NOTEBOOK_ADI}_3periyot_karsilastirma.png")

In [ ]:
# =============================================================================
# EXCEL'E KAYDET
# =============================================================================
excel_dosya = f"{RESULTS_DIR}/{NOTEBOOK_ADI}_tum_sonuclar.xlsx"

with pd.ExcelWriter(excel_dosya, engine='openpyxl') as writer:
    sonuc_df.to_excel(writer, sheet_name='Tum_Sonuclar', index=False)
    if 'pivot_rmse' in dir():
        pivot_rmse.to_excel(writer, sheet_name='RMSE')
    if 'pivot_mae' in dir():
        pivot_mae.to_excel(writer, sheet_name='MAE')
    if 'pivot_r2' in dir():
        pivot_r2.to_excel(writer, sheet_name='R2')
    if 'pivot_mape' in dir():
        pivot_mape.to_excel(writer, sheet_name='MAPE')

print(f"\n Sonuçlar kaydedildi: {excel_dosya}")
print("\n" + "=" * 80)
print(f" ÖZET - {NOTEBOOK_ADI}")
print("=" * 80)
print(f" Veri Tipi: {VERI_TIPI}")
print(f" Kategori: {KATEGORI}")
print(f" Modeller: Informer, Autoformer, TimesNet, DeepAR, TFT, TSMixer")
print(f" Periyotlar: 1 gün, 10 gün, 21 gün (HER BİRİ İÇİN AYRI EĞİTİM)")
print(f" Toplam Kayıt: {len(sonuc_df)}")
print("=" * 80)